In [5]:
import os

DATA_DIR = 'CASIA2/'
print(os.listdir(DATA_DIR))

['Tp', 'Au', 'CASIA 2 Groundtruth']


In [6]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
np.random.seed(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.python.keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout,BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.models import model_from_json
from PIL import Image, ImageChops, ImageEnhance
import PIL
import os
import itertools
from tqdm import tqdm
tf.__version__

'2.14.0'

In [9]:
def ELA(img_path, quality=90):
    TEMP = 'ela_' + 'temp.jpg'
    SCALE = 10
    original = Image.open(img_path)
    diff=""
    try:
        original.save(TEMP, quality=90)
        temporary = Image.open(TEMP)
        diff = ImageChops.difference(original, temporary)
        
    except:
        
        original.convert('RGB').save(TEMP, quality=90)
        temporary = Image.open(TEMP)
        diff = ImageChops.difference(original.convert('RGB'), temporary)
        
       
    d=diff.load()
    WIDTH, HEIGHT = diff.size
    for x in range(WIDTH):
        for y in range(HEIGHT):
            d[x, y] = tuple(k * SCALE for k in d[x, y])
#     save_path = dataset_path +'ELA_IMAGES/'
#     diff.save(save_path+'diff.png')
    return diff

In [10]:
dataset_path='CASIA2/'
path_original = 'Au/'
path_tampered = 'Tp/'
# path_mask='CASIA 2 Groundtruth/'
total_original = os.listdir(dataset_path+path_original)
total_tampered = os.listdir(dataset_path+path_tampered)
# total_mask=os.listdir(dataset_path+path_mask)

In [11]:
pristine_images = []
for i in total_original:
    pristine_images.append(dataset_path+path_original+i)
fake_images = []
for i in total_tampered:
    fake_images.append(dataset_path+path_tampered+i)

In [12]:
len(pristine_images),len(fake_images)

(7492, 5125)

In [13]:
image_size = (224,224)
output_path='../input/preprocessed-ela-images/'

In [14]:
output_path='../input/preprocessed-ela-images/'
if not os.path.exists(output_path+"resized_images/"):
#     os.makedirs(output_path+"resized_images/fake_masks/")
    os.makedirs(output_path+"resized_images/fake_images/")
    os.makedirs(output_path+"resized_images/pristine_images/")
    height = 224
    width = 224
#     p2=output_path+"resized_images/fake_masks/"
    p1=output_path+"resized_images/fake_images/"
    p3=output_path+"resized_images/pristine_images/"
    j=0
    for fake_image in tqdm(total_tampered):
        try:
            if(j%1):
                j+=1
                continue
            img=Image.open(dataset_path+path_tampered + fake_image).convert("RGB")
            img = img.resize((height, width), PIL.Image.ANTIALIAS)
            img.save(p1+fake_image)
            j+=1
        except:
            print("Encountered Invalid File : ",fake_image)
        
    j=0
    for pristine_image in tqdm(total_original):
        try:
            if(j%1):
                j+=1
                continue
            img=Image.open(dataset_path+path_original + pristine_image).convert("RGB")
            img = img.resize((height, width), PIL.Image.ANTIALIAS)
            img.save(p3+pristine_image)
            j+=1
        except:
            print("Invalid File : " ,pristine_image)
        
        
        
else:
    print('images resized,path exists')

  1%|          | 29/5125 [00:00<00:34, 149.15it/s]

Encountered Invalid File :  Tp_S_NNN_S_N_art20047_art20047_02333.tif
Encountered Invalid File :  Tp_D_NRN_M_N_nat10134_nat10124_11913.jpg
Encountered Invalid File :  Tp_D_NNN_M_B_nat00085_nat00097_11285.jpg
Encountered Invalid File :  Tp_S_CNN_S_N_txt00056_txt00056_01281.tif
Encountered Invalid File :  Tp_D_NRN_S_B_ani00036_ani00037_00156.tif
Encountered Invalid File :  Tp_S_CRN_S_N_ind00050_ind00050_10900.jpg
Encountered Invalid File :  Tp_D_NRN_S_B_ind00055_cha00040_00461.tif
Encountered Invalid File :  Tp_S_NRN_S_N_cha00080_cha00080_00400.tif
Encountered Invalid File :  Tp_S_NNN_S_N_sec20060_sec20060_01650.tif
Encountered Invalid File :  Tp_S_CRN_S_N_art00005_art00005_11792.jpg
Encountered Invalid File :  Tp_S_NNN_S_N_cha00041_cha00041_00850.tif
Encountered Invalid File :  Tp_S_CND_S_N_art00094_art00094_10404.tif
Encountered Invalid File :  Tp_D_NRD_S_B_arc20100_arc20100_02185.tif
Encountered Invalid File :  Tp_D_NRN_S_N_cha10158_ani10123_12261.jpg
Encountered Invalid File :  Tp_S_N

  1%|          | 63/5125 [00:00<00:31, 162.65it/s]

Encountered Invalid File :  Tp_S_CRN_S_N_cha10190_cha10190_12326.jpg
Encountered Invalid File :  Tp_S_NNN_S_B_ani10003_ani10003_20044.jpg
Encountered Invalid File :  Tp_S_NNN_S_N_ani20035_ani20035_02078.tif
Encountered Invalid File :  Tp_S_NNN_S_N_art20011_art20011_01817.tif
Encountered Invalid File :  Tp_D_NNN_S_B_nat00061_art00020_11442.jpg
Encountered Invalid File :  Tp_D_NRN_S_N_ani10119_ani10118_11612.jpg
Encountered Invalid File :  Tp_D_NRN_S_N_art00099_ani00005_00539.tif
Encountered Invalid File :  Tp_S_NNN_S_N_art00099_art00099_11714.jpg
Encountered Invalid File :  Tp_S_NNN_S_B_sec00028_sec00028_20116.jpg
Encountered Invalid File :  Tp_S_NRN_S_N_art20048_art20048_01854.tif
Encountered Invalid File :  Tp_S_CNN_M_N_sec00019_sec00019_10796.jpg
Encountered Invalid File :  Tp_D_NRN_S_N_cha10127_ani10123_12184.jpg
Encountered Invalid File :  Tp_S_NNN_S_N_nat00052_nat00052_11082.jpg
Encountered Invalid File :  Tp_S_CNN_S_N_arc00074_arc00074_01053.tif
Encountered Invalid File :  Tp_S_C

  2%|▏         | 99/5125 [00:00<00:30, 166.82it/s]

...

Invalid File :  Au_art_30278.jpg
Invalid File :  Au_pla_30567.jpg
Invalid File :  Au_sec_30267.jpg
Invalid File :  Au_ind_30007.jpg
Invalid File :  Au_art_20050.jpg
Invalid File :  Au_arc_00046.jpg
Invalid File :  Au_art_30564.jpg
Invalid File :  Au_cha_20030.jpg
Invalid File :  Au_sec_30187.jpg
Invalid File :  Au_arc_30453.jpg
Invalid File :  Au_arc_30357.jpg
Invalid File :  Au_pla_30418.jpg
Invalid File :  Au_cha_30375.jpg
Invalid File :  Au_ani_30232.jpg
Invalid File :  Au_nat_00046.jpg
Invalid File :  Au_nat_30398.jpg
Invalid File :  Au_pla_30591.jpg
Invalid File :  Au_cha_30162.jpg
Invalid File :  Au_sec_30057.jpg
Invalid File :  Au_cha_30033.jpg
Invalid File :  Au_nat_30327.jpg
Invalid File :  Au_nat_00051.jpg
Invalid File :  Au_ani_30443.jpg
Invalid File :  Au_nat_30400.jpg
Invalid File :  Au_pla_30269.jpg
Invalid File :  Au_nat_10002.jpg
Invalid File :  Au_nat_30253.jpg
Invalid File :  Au_cha_00003.jpg
Invalid File :  Au_sec_30037.jpg
Invalid File :  Au_arc_30619.jpg
Invalid Fi

 99%|█████████▉| 7454/7492 [00:15<00:00, 341.26it/s]

Invalid File :  Au_ani_30628.jpg
Invalid File :  Au_ani_10198.jpg
Invalid File :  Au_pla_30257.jpg
Invalid File :  Au_pla_30143.jpg
Invalid File :  Au_ani_30323.jpg
Invalid File :  Au_ind_30101.jpg
Invalid File :  Au_arc_30069.jpg
Invalid File :  Au_pla_30019.jpg
Invalid File :  Au_sec_00079.jpg
Invalid File :  Au_arc_30566.jpg
Invalid File :  Au_nat_30101.jpg
Invalid File :  Au_sec_30620.jpg
Invalid File :  Au_pla_30294.jpg
Invalid File :  Au_nat_30391.jpg
Invalid File :  Au_ind_30116.jpg
Invalid File :  Au_pla_30443.jpg
Invalid File :  Au_ani_30056.jpg
Invalid File :  Au_art_30248.jpg
Invalid File :  Au_cha_10116.jpg
Invalid File :  Au_ani_30334.jpg
Invalid File :  Au_ani_30608.jpg
Invalid File :  Au_ani_30058.jpg
Invalid File :  Au_arc_20012.jpg
Invalid File :  Au_art_30127.jpg
Invalid File :  Au_art_30573.jpg
Invalid File :  Au_cha_30597.jpg
Invalid File :  Au_art_30306.jpg
Invalid File :  Au_art_30307.jpg
Invalid File :  Au_arc_10112.jpg
Invalid File :  Au_ani_30190.jpg
Invalid Fi

100%|██████████| 7492/7492 [00:15<00:00, 474.29it/s]

Invalid File :  Au_nat_30067.jpg
Invalid File :  Au_cha_30259.jpg


In [10]:
resized_fake_image_path=output_path+"resized_images/fake_images/"
resized_pristine_image_path=output_path+"resized_images/pristine_images/"
resized_fake_image=os.listdir(resized_fake_image_path)
resized_pristine_image=os.listdir(resized_pristine_image_path)

In [11]:
len(resized_fake_image), len(resized_pristine_image)

(5122, 7491)

In [12]:
ela_images_path=output_path+'ELA_IMAGES/'
ela_real=ela_images_path+'Au/'
ela_fake=ela_images_path+'Tp/'
if not os.path.exists(ela_images_path):
    os.makedirs(ela_images_path)
    os.mkdir(ela_real)
    os.mkdir(ela_fake)
    j=0
    for i in tqdm(resized_fake_image):
        ELA(resized_fake_image_path+i).save(ela_fake+i)
        j+=1
        if(j==15000):
            break
    j=0
    for i in tqdm(resized_pristine_image):
        ELA(resized_pristine_image_path+i).save(ela_real+i)
        j+=1
        if(j==15000):
            break
else:
    print('Images are already converted to ELA')

Images are already converted to ELA


In [13]:
X=[]
Y=[]
j=0
for file in tqdm(os.listdir(ela_real)):
    img=Image.open(ela_real+file)
    img=np.array(img)
    X.append(img)
    Y.append(0)
    j+=1
    if(j==15000):
        break
j=0
for file in tqdm(os.listdir(ela_fake)):
    img=Image.open(ela_fake+file)
    img=np.array(img)
    X.append(img)
    Y.append(1)
    j+=1
    if(j==15000):
        break

100%|█████████████████████████████████████████████████████████████████████████████| 5123/5123 [00:33<00:00, 153.73it/s]


In [14]:
X=np.array(X)
X.shape

(12614, 224, 224, 3)

In [15]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

x_train, x_dev, y_train, y_dev = train_test_split(X, Y, test_size=0.2, random_state=133,shuffle=True)
y_train=to_categorical(y_train,2)
y_dev=to_categorical(y_dev,2)
print(len(x_train), len(y_train))
print(len(x_dev), len(y_dev))

10091 10091
2523 2523


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, AvgPool2D, MaxPool2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
def CNN():
    model = Sequential()
    model.add(Conv2D(filters=128,kernel_size=(5,5),activation='relu',input_shape=(224,224,3)))
    #model.add(MaxPool2D(pool_size=(2,2)))
    model.add(AvgPool2D(pool_size=(2,2)))
    model.add(Conv2D(filters=256,kernel_size=(5,5),activation='relu'))
    #model.add(MaxPool2D(pool_size=(2,2)))
    model.add(AvgPool2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(units=64,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=16,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=2,activation='softmax'))
    return model

In [17]:
model1=CNN()
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 128)     9728      
                                                                 
 average_pooling2d (Average  (None, 110, 110, 128)     0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 106, 256)     819456    
                                                                 
 average_pooling2d_1 (Avera  (None, 53, 53, 256)       0         
 gePooling2D)                                                    
                                                                 
 flatten (Flatten)           (None, 719104)            0         
                                                                 
 dense (Dense)               (None, 64)                4

In [18]:
epochs = 30
batch_size = 32
init_lr = 1e-4

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = init_lr, decay = init_lr/epochs)

model1.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [19]:
import keras

early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=6,verbose=1,restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.22, patience = 6, verbose = 1, 
                                              min_delta = 0.0001,min_lr=0.0001)

In [ ]:
hist = model1.fit(x_train,y_train,
                 epochs = epochs,
                validation_data = (x_dev,y_dev),
                callbacks = [early_stop,reduce_lr],
                verbose=1,shuffle=True)

Epoch 1/30
 50/316 [===>..........................] - ETA: 34:30 - loss: 1.7044 - accuracy: 0.5756

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title("Accuracy")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','test'])
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("Loss")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','test'])
plt.show()

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
   
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
Y_pred = model1.predict(x_dev)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(y_dev,axis = 1) 
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
plot_confusion_matrix(confusion_mtx, classes = range(2))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_true, Y_pred_classes))